In [10]:
import json
from tqdm import tqdm 
output_path = '/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.all_risk.v9.1'
result_list = []
with open(output_path) as frobj:
    for line in tqdm(frobj):
        content = json.loads(line.strip())
        result_list.append(content)
        
        
        

2071401it [01:39, 20901.43it/s]


In [84]:
with open('/data/albert.xht/xiaodao/query_risk_v10/query_risk_itag.json', 'w') as fwobj:
    # fwobj.write("&&&".join(['content'])+'\n')
    with open('/data/albert.xht/xiaodao/query_risk_v10/query_risk_corpus.json.select', 'r') as frobj:
        for idx, line in enumerate(frobj):
            content = json.loads(line.strip())
            inputs = [content['text']]
            fwobj.write("&&&".join(inputs)+'\n')
    with open('/data/albert.xht/xiaodao/query_risk_v10/biake_qa_web_text_zh_train.json.offensive.all', 'r') as frobj:
        for idx, line in enumerate(frobj):
            content = json.loads(line.strip())
            inputs = [content['text']]
            fwobj.write("&&&".join(inputs)+'\n')
    

In [11]:

embed_list = []
text_list = []
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/biake_qa_web_text_zh_train.json.embed') as frobj:
    for line in tqdm(frobj):
        content = json.loads(line.strip())
        embed_list.append(content['feat'])
        text_list.append(content['text'])


2071401it [08:05, 4270.41it/s]


In [13]:
from torch.utils.data import Dataset
from torch.autograd import Variable
import torch.nn as nn


class LogitClipLoss(nn.Module):
    def __init__(self, device, t=1.0):
        super(LogitClipLoss, self).__init__()
        self.device = device
        self.t = t

    def forward(self, x, target, reduction):
        norms = torch.norm(x, p=2, dim=-1, keepdim=True) + 1e-7
        logit_norm = torch.div(x, norms) / self.t
        clip = (norms > self.t).expand(-1, x.shape[-1])
        logit_clip = torch.where(clip, logit_norm, x)
        return F.cross_entropy(logit_clip, target, reduction=reduction)

class data_generator_embed(Dataset):
    def __init__(self, data, label):
        
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

    @staticmethod
    def collate(examples):
        batch_embed, batch_label = [], []
        for item in examples: # batch_size
            embed, label = item
            batch_label.append(label)
            batch_embed.append(embed)

        batch_embed = torch.tensor(batch_embed).float()
        batch_label = torch.tensor(batch_label).long()#RoBERTa 不需要NSP
         
        return [batch_embed, batch_label]
    
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
import logging
import warnings
from tqdm import tqdm

class EmbedLinear(nn.Module):
    def __init__(self, input_dim, hidden_size, dropout_prob, num_labels, device):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.dropout_prob = dropout_prob
        self.num_labels = num_labels
        
        self.dense = nn.Linear(input_dim, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_prob)
        
        self.out_proj = nn.Linear(self.hidden_size, self.num_labels)
        self.device = device

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        last_rep = torch.tanh(x)
        last_rep = self.dropout(last_rep)
        logits = self.out_proj(last_rep)
        return logits
    
    def compile(self, method="logistic", optimizer="Adam", learning_rate=0.001, momentum=0.0):
        """Wrapper to set the activation, loss and the optimizer.
        Parameters:
        ----------
        method (str) : regression, logistic or multiclass
        optimizer (str): SGD, Adam, or RMSprop
        """
        if method == 'regression':
            self.activation, self.criterion = None, F.mse_loss
        if method == 'logistic':
            self.activation, self.criterion = F.sigmoid, F.binary_cross_entropy
        if method == 'multiclass':
            self.activation, self.criterion = F.softmax, F.cross_entropy
        if method == 'multiclass_logitclip':
            self.activation, self.criterion = F.softmax, LogitClipLoss(self.device, tau=1.5)

        if optimizer == "Adam":
            self.optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        if optimizer == "RMSprop":
            self.optimizer = torch.optim.RMSprop(self.parameters(), lr=learning_rate)
        if optimizer == "SGD":
            self.optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate, momentum=momentum)

        self.method = method
    
    def fit(self, data, label, epoch, batch_size):
        train_data = data_generator_embed(
                    data, label
                    )
        collate_fn = train_data.collate
        print(len(train_data))
        train_loader = torch.utils.data.DataLoader(train_data, 
                                                   batch_size=batch_size, collate_fn=collate_fn,
                                                shuffle=True, drop_last=True)
        
        for epoch in tqdm(range(epoch)):
            net = self.train()
            for i, (embed, target) in (enumerate(train_loader)):
                embed, target = embed.to(self.device), target.to(self.device)

                self.optimizer.zero_grad()
                
                y_pred =  net(embed)
                loss = self.criterion(y_pred, target)
                loss.backward()
                self.optimizer.step()
    
    def predict_proba(self, data, batch_size):
        """Predict predict probability for dataset.
        This method will only work with method logistic/multiclass
        Parameters:
        ----------
        dataset (dict): dictionary with the testing dataset -
        X_wide_test, X_deep_test, target
        Returns:
        --------
        array-like with the probability for dataset.
        """
        queue = []
        pred_list = []
        net = self.eval()
        for d in data:
            queue.append(d)
            if np.mod(len(queue), batch_size) == 0:
                embed = torch.tensor(queue).float()
                embed = embed.to(self.device)
                with torch.no_grad():
                    pred = net(embed)
                    probs = nn.Softmax(dim=-1)(pred).cpu().data.numpy()
                pred_list.extend(probs)
                queue = []
        if queue:
            embed = torch.tensor(queue).float()
            embed = embed.to(self.device)
            with torch.no_grad():
                pred = net(embed)
                probs = nn.Softmax(dim=-1)(pred).cpu().data.numpy()
            pred_list.extend(probs)
        return np.array(pred_list)


In [39]:
from torch.utils.data import Dataset
from torch.autograd import Variable
import torch.nn as nn


class LogitClipLoss(nn.Module):
    def __init__(self, device, t=1.0):
        super(LogitClipLoss, self).__init__()
        self.device = device
        self.t = t

    def forward(self, x, target, reduction='mean'):
        norms = torch.norm(x, p=2, dim=-1, keepdim=True) + 1e-7
        logit_norm = torch.div(x, norms) / self.t
        clip = (norms > self.t).expand(-1, x.shape[-1])
        logit_clip = torch.where(clip, logit_norm, x)
        return F.cross_entropy(logit_clip, target, reduction=reduction)

class data_generator_wide_and_deep(Dataset):
    def __init__(self, data, feat, label):
        
        self.data = data
        self.feat = feat
        self.label = label
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.feat[idx], self.label[idx]

    @staticmethod
    def collate(examples):
        batch_embed, batch_feat, batch_label = [], [], []
        for item in examples: # batch_size
            embed, feat, label = item
            batch_label.append(label)
            batch_embed.append(embed)
            batch_feat.append(feat)
            
        batch_embed = torch.tensor(batch_embed).float()
        batch_label = torch.tensor(batch_label).long()#RoBERTa 不需要NSP
        batch_feat = torch.tensor(batch_feat).float()
         
        return [batch_embed, batch_feat, batch_label]
    
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
import logging
import warnings
from tqdm import tqdm

class EmbedLinear_wide_and_deep(nn.Module):
    def __init__(self, input_dim, hidden_size, dropout_prob, num_labels, device):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.dropout_prob = dropout_prob
        self.num_labels = num_labels
        
        self.dense = nn.Linear(input_dim, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_prob)
        
        self.out_proj = nn.Linear(self.hidden_size, self.num_labels)
        self.device = device

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        last_rep = torch.tanh(x)
        last_rep = self.dropout(last_rep)
        logits = self.out_proj(last_rep)
        return logits
    
    def compile(self, method="logistic", optimizer="Adam", learning_rate=0.001, momentum=0.0):
        """Wrapper to set the activation, loss and the optimizer.
        Parameters:
        ----------
        method (str) : regression, logistic or multiclass
        optimizer (str): SGD, Adam, or RMSprop
        """
        if method == 'regression':
            self.activation, self.criterion = None, F.mse_loss
        if method == 'logistic':
            self.activation, self.criterion = F.sigmoid, F.binary_cross_entropy
        if method == 'multiclass':
            self.activation, self.criterion = F.softmax, F.cross_entropy
        if method == 'multiclass_logitclip':
            self.activation, self.criterion = F.softmax, LogitClipLoss(self.device, tau=1.5)

        if optimizer == "Adam":
            self.optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        if optimizer == "RMSprop":
            self.optimizer = torch.optim.RMSprop(self.parameters(), lr=learning_rate)
        if optimizer == "SGD":
            self.optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate, momentum=momentum)

        self.method = method
    
    def fit(self, data, feat, label, epoch, batch_size):
        train_data = data_generator_wide_and_deep(
                    data, feat, label
                    )
        collate_fn = train_data.collate
        print(len(train_data))
        train_loader = torch.utils.data.DataLoader(train_data, 
                                                   batch_size=batch_size, collate_fn=collate_fn,
                                                shuffle=True, drop_last=True)
        
        for epoch in tqdm(range(epoch)):
            net = self.train()
            for i, (embed, feat, target) in (enumerate(train_loader)):
                embed, feat, target = embed.to(self.device), feat.to(self.device), target.to(self.device)
                embed = torch.cat([feat, embed], dim=-1)
                
                self.optimizer.zero_grad()
                
                y_pred =  net(embed)
                loss = self.criterion(y_pred, target)
                loss.backward()
                self.optimizer.step()
    
    def predict_proba(self, data, feat, batch_size):
        """Predict predict probability for dataset.
        This method will only work with method logistic/multiclass
        Parameters:
        ----------
        dataset (dict): dictionary with the testing dataset -
        X_wide_test, X_deep_test, target
        Returns:
        --------
        array-like with the probability for dataset.
        """
        queue_d = []
        queue_f = []
        pred_list = []
        net = self.eval()
        for d, f in zip(data, feat):
            queue_d.append(d)
            queue_f.append(f)
            if np.mod(len(queue_d), batch_size) == 0:
                embed = torch.tensor(queue_d).float()
                embed = embed.to(self.device)
                feat = torch.tensor(queue_f).float()
                feat = feat.to(self.device)
                embed = torch.cat([feat, embed], dim=-1)
                with torch.no_grad():
                    pred = net(embed)
                    probs = nn.Softmax(dim=-1)(pred).cpu().data.numpy()
                pred_list.extend(probs)
                queue_d = []
                queue_f = []
        if queue_d:
            embed = torch.tensor(queue_d).float()
            embed = embed.to(self.device)
            feat = torch.tensor(queue_f).float()
            feat = feat.to(self.device)
            embed = torch.cat([feat, embed], dim=-1)
            with torch.no_grad():
                pred = net(embed)
                probs = nn.Softmax(dim=-1)(pred).cpu().data.numpy()
            pred_list.extend(probs)
        return np.array(pred_list)


In [26]:
import warnings

import numpy as np
from numba import njit
from scipy.special import softmax


@njit
def each_evidence(y_, f, fh, v, s, vh, N, D):
    """
    compute the maximum evidence for each class
    """
    epsilon = 1e-5
    alpha = 1.0
    beta = 1.0
    lam = alpha / beta
    tmp = (vh @ (f @ np.ascontiguousarray(y_)))
    for _ in range(11):
        # should converge after at most 10 steps
        # typically converge after two or three steps
        gamma = (s / (s + lam)).sum()
        # A = v @ np.diag(alpha + beta * s) @ v.transpose() # no need to compute A
        # A_inv = v @ np.diag(1.0 / (alpha + beta * s)) @ v.transpose() # no need to compute A_inv
        m = v @ (tmp * beta / (alpha + beta * s))
        alpha_de = (m * m).sum()
        alpha = gamma / (alpha_de + epsilon)
        beta_de = ((y_ - fh @ m) ** 2).sum()
        beta = (N - gamma) / (beta_de + epsilon)
        new_lam = alpha / beta
        if np.abs(new_lam - lam) / lam < 0.01:
            break
        lam = new_lam
    evidence = D / 2.0 * np.log(alpha) \
               + N / 2.0 * np.log(beta) \
               - 0.5 * np.sum(np.log(alpha + beta * s)) \
               - beta / 2.0 * (beta_de + epsilon) \
               - alpha / 2.0 * (alpha_de + epsilon) \
               - N / 2.0 * np.log(2 * np.pi)
    return evidence / N, alpha, beta, m


# use pseudo data to compile the function
# D = 20, N = 50
f_tmp = np.random.randn(20, 50).astype(np.float64)
each_evidence(np.random.randint(0, 2, 50).astype(np.float64), f_tmp, f_tmp.transpose(), np.eye(20, dtype=np.float64), np.ones(20, dtype=np.float64), np.eye(20, dtype=np.float64), 50, 20)


@njit
def truncated_svd(x):
    u, s, vh = np.linalg.svd(x.transpose() @ x)
    s = np.sqrt(s)
    u_times_sigma = x @ vh.transpose()
    k = np.sum((s > 1e-10) * 1)  # rank of f
    s = s.reshape(-1, 1)
    s = s[:k]
    vh = vh[:k]
    u = u_times_sigma[:, :k] / s.reshape(1, -1)
    return u, s, vh
truncated_svd(np.random.randn(20, 10).astype(np.float64))


class LogME(object):
    def __init__(self, regression=False):
        """
            :param regression: whether regression
        """
        self.regression = regression
        self.fitted = False
        self.reset()

    def reset(self):
        self.num_dim = 0
        self.alphas = []  # alpha for each class / dimension
        self.betas = []  # beta for each class / dimension
        # self.ms.shape --> [C, D]
        self.ms = []  # m for each class / dimension

    def _fit_icml(self, f: np.ndarray, y: np.ndarray):
        """
        LogME calculation proposed in the ICML 2021 paper
        "LogME: Practical Assessment of Pre-trained Models for Transfer Learning"
        at http://proceedings.mlr.press/v139/you21b.html
        """
        fh = f
        f = f.transpose()
        D, N = f.shape
        v, s, vh = np.linalg.svd(f @ fh, full_matrices=True)

        evidences = []
        self.num_dim = y.shape[1] if self.regression else int(y.max() + 1)
        for i in range(self.num_dim):
            y_ = y[:, i] if self.regression else (y == i).astype(np.float64)
            evidence, alpha, beta, m = each_evidence(y_, f, fh, v, s, vh, N, D)
            evidences.append(evidence)
            self.alphas.append(alpha)
            self.betas.append(beta)
            self.ms.append(m)
        self.ms = np.stack(self.ms)
        return np.mean(evidences)

    def _fit_fixed_point(self, f: np.ndarray, y: np.ndarray):
        """
        LogME calculation proposed in the arxiv 2021 paper
        "Ranking and Tuning Pre-trained Models: A New Paradigm of Exploiting Model Hubs"
        at https://arxiv.org/abs/2110.10545
        """
        N, D = f.shape  # k = min(N, D)
        if N > D: # direct SVD may be expensive
            u, s, vh = truncated_svd(f)
        else:
            u, s, vh = np.linalg.svd(f, full_matrices=False)
        # u.shape = N x k
        # s.shape = k
        # vh.shape = k x D
        s = s.reshape(-1, 1)
        sigma = (s ** 2)

        evidences = []
        self.num_dim = y.shape[1] if self.regression else int(y.max() + 1)
        for i in range(self.num_dim):
            y_ = y[:, i] if self.regression else (y == i).astype(np.float64)
            y_ = y_.reshape(-1, 1)
            x = u.T @ y_  # x has shape [k, 1], but actually x should have shape [N, 1]
            x2 = x ** 2
            res_x2 = (y_ ** 2).sum() - x2.sum()  # if k < N, we compute sum of xi for 0 singular values directly

            alpha, beta = 1.0, 1.0
            for _ in range(11):
                t = alpha / beta
                gamma = (sigma / (sigma + t)).sum()
                m2 = (sigma * x2 / ((t + sigma) ** 2)).sum()
                res2 = (x2 / ((1 + sigma / t) ** 2)).sum() + res_x2
                alpha = gamma / (m2 + 1e-5)
                beta = (N - gamma) / (res2 + 1e-5)
                t_ = alpha / beta
                evidence = D / 2.0 * np.log(alpha) \
                           + N / 2.0 * np.log(beta) \
                           - 0.5 * np.sum(np.log(alpha + beta * sigma)) \
                           - beta / 2.0 * res2 \
                           - alpha / 2.0 * m2 \
                           - N / 2.0 * np.log(2 * np.pi)
                evidence /= N
                if abs(t_ - t) / t <= 1e-3:  # abs(t_ - t) <= 1e-5 or abs(1 / t_ - 1 / t) <= 1e-5:
                    break
            evidence = D / 2.0 * np.log(alpha) \
                       + N / 2.0 * np.log(beta) \
                       - 0.5 * np.sum(np.log(alpha + beta * sigma)) \
                       - beta / 2.0 * res2 \
                       - alpha / 2.0 * m2 \
                       - N / 2.0 * np.log(2 * np.pi)
            evidence /= N
            m = 1.0 / (t + sigma) * s * x
            m = (vh.T @ m).reshape(-1)
            evidences.append(evidence)
            self.alphas.append(alpha)
            self.betas.append(beta)
            self.ms.append(m)
        self.ms = np.stack(self.ms)
        return np.mean(evidences)

    _fit = _fit_fixed_point

    def fit(self, f: np.ndarray, y: np.ndarray):
        """
        :param f: [N, F], feature matrix from pre-trained model
        :param y: target labels.
            For classification, y has shape [N] with element in [0, C_t).
            For regression, y has shape [N, C] with C regression-labels
        :return: LogME score (how well f can fit y directly)
        """
        if self.fitted:
            warnings.warn('re-fitting for new data. old parameters cleared.')
            self.reset()
        else:
            self.fitted = True
        f = f.astype(np.float64)
        if self.regression:
            y = y.astype(np.float64)
            if len(y.shape) == 1:
                y = y.reshape(-1, 1)
        return self._fit(f, y)

    def predict(self, f: np.ndarray):
        """
        :param f: [N, F], feature matrix
        :return: prediction, return shape [N, X]
        """
        if not self.fitted:
            raise RuntimeError("not fitted, please call fit first")
        f = f.astype(np.float64)
        logits = f @ self.ms.T
        if self.regression:
            return logits
        return np.argmax(logits, axis=-1)
    
    def predict_prob(self, f: np.ndarray):
        """
        :param f: [N, F], feature matrix
        :return: prediction, return shape [N, X]
        """
        if not self.fitted:
            raise RuntimeError("not fitted, please call fit first")
        f = f.astype(np.float64)
        logits = f @ self.ms.T
        if self.regression:
            return logits
        return softmax(logits, axis=-1)
    

In [14]:
import numpy as np
result_matrix = []
result_item = []
result_embed = []
result_text = []
for idx, item in tqdm(enumerate(result_list)):
    p = []
    for key in ['senti', 'bias', 'ciron', 'offensive', 'teenager', 'senti_query']:
        if item['score_list'][key][0][1] >= 0.5:
            p.append(1)
        else:
            p.append(0)
    if item['score_list']['query_risk'][0][1] >= 0.5:
        p.append(1)
    else:
        p.append(0)
    if item['score_list']['query_risk'][1][1] >= 0.5:
        p.append(1)
    else:
        p.append(0)
    if sum(p) <= 1:
        continue
    result_matrix.append(p)
    result_item.append(item)
    result_embed.append(embed_list[idx])
    result_text.append(text_list[idx])
    

# result_matrix = np.array(result_matrix)
# votes = np.sum(result_matrix, axis=-1)
# labels = np.array(votes >= 3).astype(np.int)
# sum(labels)

2071401it [00:07, 280165.95it/s]


In [ ]:
import xgboost as xgb
class make_model():
    def __init__(self,param,num_round=300):
        self.param=param
        self.num_round=num_round
    def fit(self,gen_data):
        iteration = 0
        
        for df in gen_data:
            dtrain = xgb.DMatrix(np.array(df[features]), label=df['label'])
            if iteration ==0:
                model = xgb.Booster(self.param, [dtrain])
            model = xgb.train(self.param,dtrain,num_boost_round=1, xgb_model=model)
            iteration += 1
            
        self.model_=model
    def predict(self,X):
        dtest=xgb.DMatrix(X)
        return self.model_.predict(dtest)>0.5 # use argmax in non-binary classification
parameters = {'max_depth':5, "booster":"gbtree"} # parameters to tune, see xgboost doc. Can be used to make boosted trees or Random Forests.
model = make_model(parameters) 
model.fit(gen_data)
xgb.plot_importance(model.model_)

In [15]:

votes = np.sum(result_matrix, axis=-1)
labels = np.array(votes >= 3).astype(np.int)
epoch = 1
batch_size = 512

model_list = []

for i in range(5):
    
    device = "cuda:1"
    model = EmbedLinear(input_dim=384, hidden_size=128, dropout_prob=0.1, num_labels=2, device=device).to(device)
    model.compile(method="multiclass", optimizer="Adam", learning_rate=1e-4, momentum=0.0)    
    
    print('===start====', i)
    result = model.fit(result_embed, labels, epoch=epoch, batch_size=batch_size)
    probs = model.predict_proba(result_embed, batch_size=batch_size)
    print(probs.shape)
    prev_labels = np.argmax(probs, axis=-1)
    diff = (prev_labels != np.array(labels)).sum()
    print(diff / prev_labels.shape[0])
    labels = prev_labels
    
    model_list.append(model)
    
    

  0%|          | 0/1 [00:00<?, ?it/s]

===start==== 0
772017


  0%|          | 0/1 [00:00<?, ?it/s]

(772017, 2)
0.1335566444780361
===start==== 1
772017


  0%|          | 0/1 [00:00<?, ?it/s]

...

(772017, 2)
0.018408921047075387
===start==== 3
772017


  0%|          | 0/1 [00:00<?, ?it/s]

(772017, 2)
0.015109770898827358
===start==== 4
772017


100%|██████████| 1/1 [00:14<00:00, 14.87s/it]


(772017, 2)
0.010936287672421722


In [40]:

votes = np.sum(result_matrix, axis=-1)
labels = np.array(votes >= 3).astype(np.int)
epoch = 1
batch_size = 512

model_list_wide_and_deep = []

for i in range(5):
    
    device = "cuda:1"
    model = EmbedLinear_wide_and_deep(input_dim=384+8, hidden_size=128, dropout_prob=0.1, num_labels=2, device=device).to(device)
    model.compile(method="multiclass", optimizer="Adam", learning_rate=1e-4, momentum=0.0)    
    
    print('===start====', i)
    result = model.fit(result_embed, np.array(result_matrix), labels, epoch=epoch, batch_size=batch_size)
    probs = model.predict_proba(result_embed, np.array(result_matrix), batch_size=batch_size)
    print(probs.shape)
    prev_labels = np.argmax(probs, axis=-1)
    diff = (prev_labels != np.array(labels)).sum()
    print(diff / prev_labels.shape[0])
    labels = prev_labels
    
    model_list_wide_and_deep.append(model)
    
    

  0%|          | 0/1 [00:00<?, ?it/s]

===start==== 0
772017


100%|██████████| 1/1 [00:17<00:00, 17.05s/it]


(772017, 2)
0.03231664587696903
===start==== 1


  0%|          | 0/1 [00:00<?, ?it/s]

...

(772017, 2)
0.013083908774029587
===start==== 4


  0%|          | 0/1 [00:00<?, ?it/s]

772017


100%|██████████| 1/1 [00:16<00:00, 16.93s/it]


(772017, 2)
0.009998484489331194


In [64]:
probs = model_list[1].predict_proba(result_embed, batch_size=batch_size)

In [65]:
probs_wide_and_deep = model_list_wide_and_deep[0].predict_proba(result_embed, np.array(result_matrix), batch_size=batch_size)

In [79]:
black = []
for prob, prob_wide_and_deep, item in zip(probs, probs_wide_and_deep, result_item):
    if prob[1] >= 0.9 and prob_wide_and_deep[1] > 0.9:
        black.append((prob, item))
    

In [81]:
black[10:20]

[(array([0.03497289, 0.9650271 ], dtype=float32),
  {'text': '追赶小偷致其跳河溺亡应如何处置？',
   'topic': ['法律'],
   'score_list': {'senti_query': [['负向', 0.9976511597633362],
     ['中性', 0.0022351674269884825],
     ['正向', 0.00011369012645445764]],
    'senti': [['负向', 0.9974364638328552], ['正向', 0.0025636202190071344]],
    'bias': [['偏见', 0.25562426447868347], ['正常', 0.7443757057189941]],
    'ciron': [['讽刺', 0.2297625094652176], ['正常', 0.7702375054359436]],
    'intent': [['主观评价/比较/判断', 0.0007977944333106279],
     ['寻求建议/帮助', 0.9991441965103149],
     ['其它', 5.80373452976346e-05]],
    'offensive': [['冒犯', 0.9323585629463196], ['正常', 0.0676414966583252]],
    'query_risk': [['风险', 0.9970659613609314],
     ['个人信息', 0.0005186186754144728],
     ['正常', 0.0024155047722160816]],
    'teenager': [['不良', 0.36774078011512756], ['正常', 0.6322592496871948]]}}),
 (array([3.8990277e-04, 9.9961013e-01], dtype=float32),
  {'text': '如何看待部分中国人认为欧洲收留叙利亚难民是圣母行为？',
   'topic': ['宗教'],
   'score_list': {'senti_qu

In [26]:
torch.save(model.state_dict(), '/data/albert.xht/raw_chat_corpus/topic_classification_v4/embed_linear_small.bin')


In [61]:
black = []
white = []
for text, label, prob, resp in zip(result_text, labels, probs, result_item):
    if prob[1] > 0.9:
        black.append((label, prob, resp, text))
    elif prob[0] > 0.9 and resp['score_list']['query_risk'][0][-1] > 0.5 and resp['score_list']['query_risk'][0][-1] < 0.7:
        white.append((label, prob, resp, text))

# labels[0:10]

In [32]:
logme = LogME(regression=False)



In [53]:
a = np.random.permutation(np.array(range(10)))
np.array(range(10))[a], np.array(range(10)), a

(array([0, 8, 2, 3, 1, 9, 5, 6, 7, 4]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([0, 8, 2, 3, 1, 9, 5, 6, 7, 4]))

In [54]:
votes = np.sum(result_matrix, axis=-1)
labels = np.array(votes >= 3).astype(np.int)
result_embed_np = np.array(result_embed)
print(labels.sum(), result_embed_np.shape, labels.shape)
logme_list = []

for i in tqdm(range(10)):
    # f has shape of [N, D], y has shape [N]
    logme = LogME(regression=False)
    
    score = logme.fit(result_embed_np, labels)
    new_labels = logme.predict(result_embed_np)
    diff = (new_labels != np.array(labels)).sum()
    print(diff / new_labels.shape[0], score, new_labels.sum())
    labels = new_labels
    logme_list.append(logme)
    

  0%|          | 0/10 [00:00<?, ?it/s]

162260 (772017, 384) (772017,)


 10%|█         | 1/10 [00:28<04:18, 28.71s/it]

0.1297950692795625 -0.2483103892726054 118898


 20%|██        | 2/10 [01:08<04:15, 31.90s/it]

...

0.01321214429215937 0.7459536845734029 11024


 90%|█████████ | 9/10 [05:42<00:36, 36.03s/it]

0.010585259132894742 0.9882790021219723 2852


100%|██████████| 10/10 [06:13<00:00, 37.39s/it]

0.0036942191687488746 1.485124312146819 0


In [142]:
senti_input = []
senti_label = []
with open('/data/albert.xht/sentiment/senti_copr.json.embed') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        senti_input.append(content['feat'])
        senti_label.append(int(content['label']))

logme_senti_list = []

senti_feat_np = np.array(senti_input)
senti_label_np = np.array(senti_label)

for i in tqdm(range(10)):
    # f has shape of [N, D], y has shape [N]
    logme = LogME(regression=False)
    
    score = logme.fit(senti_feat_np, senti_label_np)
    new_labels = logme.predict(senti_feat_np)
    diff = (new_labels != np.array(senti_label_np)).sum()
    print(diff / new_labels.shape[0], score, new_labels.sum())
    senti_label_np = new_labels
    logme_senti_list.append(logme)
    

 10%|█         | 1/10 [00:29<04:23, 29.27s/it]

0.12099683025467264 -0.3656026754108087 4645


 20%|██        | 2/10 [00:54<03:43, 27.91s/it]

0.017925456334025577 -0.10830362679650674 4599


 30%|███       | 3/10 [01:30<03:32, 30.41s/it]

...

0.0019674281342223193 -0.07438947745710067 4518


 90%|█████████ | 9/10 [04:35<00:32, 32.02s/it]

0.0018581265712099683 -0.07359154878531099 4511


100%|██████████| 10/10 [05:10<00:00, 31.02s/it]

0.001530221882172915 -0.07276646570251749 4505


In [145]:
senti_dev = []
senti_dev_label = []
with open('/data/albert.xht/sentiment/senti_copr_dev.json.embed') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        senti_dev.append(content['feat'])
        senti_dev_label.append(int(content['label']))

dev_predict_label = logme_senti_list[-1].predict(np.array(senti_dev))
(dev_predict_label == np.array(senti_dev_label)).sum()/dev_predict_label.shape[0]

0.833041958041958

In [138]:
senti_dev_label[0:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [155]:
labels = logme_list[1].predict_prob(result_embed_np)


In [157]:
black = []
white = []
for text, label, resp in zip(result_text, labels, result_item):
    if label[1] > 0.6:
        black.append((label, text, resp))
    elif label[1] < 0.4 and resp['score_list']['query_risk'][0][-1] > 0.5 and resp['score_list']['query_risk'][0][-1] < 0.7:
        white.append((label, text, resp))


In [159]:
white

[(array([0.61923084, 0.38076916]),
  '啤酒加海鲜是导致痛风的真凶吗？',
  {'text': '啤酒加海鲜是导致痛风的真凶吗？',
   'topic': ['健康'],
   'score_list': {'senti_query': [['负向', 0.9878260493278503],
     ['中性', 0.011836989782750607],
     ['正向', 0.00033692491706460714]],
    'senti': [['负向', 0.9924633502960205], ['正向', 0.007536637131124735]],
    'bias': [['偏见', 0.03395457565784454], ['正常', 0.9660454392433167]],
    'ciron': [['讽刺', 0.049804456532001495], ['正常', 0.9501956105232239]],
    'intent': [['主观评价/比较/判断', 0.00010593750630505383],
     ['寻求建议/帮助', 0.9996864795684814],
     ['其它', 0.00020762969506904483]],
    'offensive': [['冒犯', 0.12625817954540253], ['正常', 0.8737418055534363]],
    'query_risk': [['风险', 0.5278050899505615],
     ['个人信息', 4.180899486527778e-05],
     ['正常', 0.4721531271934509]],
    'teenager': [['不良', 0.03140368312597275], ['正常', 0.968596339225769]]}}),
 (array([0.64696887, 0.35303113]),
  '如果咨询师忽然去世，这场「心理咨询」还有哪些后续工作需要处理？',
  {'text': '如果咨询师忽然去世，这场「心理咨询」还有哪些后续工作需要处理？',
   'topic': ['心理健康'],

In [7]:
import json
black = []
with open('/data/albert.xht/xiaodao/query_risk_v10/biake_qa_web_text_zh_train.json.offensive.all.black.v10.1', 'w') as fwobj:
    with open('/data/albert.xht/xiaodao/query_risk_v10/biake_qa_web_text_zh_train.json.offensive.all.v10.1') as frobj:
        for line in frobj:
            content = json.loads(line.strip())
            if content['score_list']['offensive'][0][1] >= 0.8:
                black.append(content)
                fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')
                

[{'text': '为什么女司机被打那么多人同情「弱者」而选择性忽视「弱者」的过错可能导致的严重后果？',
  'topic': ['风险'],
  'score_list': {'senti_query': [['负向', 0.9979752898216248],
    ['中性', 0.0019358349964022636],
    ['正向', 8.883338887244463e-05]],
   'senti': [['负向', 0.9958634376525879], ['正向', 0.004136575851589441]],
   'bias': [['偏见', 0.27606531977653503], ['正常', 0.7239347100257874]],
   'ciron': [['讽刺', 0.30140814185142517], ['正常', 0.6985918879508972]],
   'intent': [['主观评价/比较/判断', 0.9471869468688965],
    ['寻求建议/帮助', 0.05241207033395767],
    ['其它', 0.0004009997355751693]],
   'offensive': [['冒犯', 0.8063883185386658], ['正常', 0.19361165165901184]],
   'query_risk': [['风险', 0.9768148064613342],
    ['个人信息', 2.531747850298416e-05],
    ['正常', 0.023159895092248917]],
   'teenager': [['不良', 0.08330792188644409], ['正常', 0.9166920781135559]]}},
 {'text': '哪些行为会使男性显得俗不可耐？',
  'topic': ['风险'],
  'score_list': {'senti_query': [['负向', 0.9975232481956482],
    ['中性', 0.0022904882207512856],
    ['正向', 0.00018620019545778632]],
   'sen

In [48]:
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/logme_black.json', 'w') as fwobj:
    for l in black:
        content = {
            'text':l[0],
            'label':['风险']
        }
        fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')

In [63]:
with open('/data/albert.xht/raw_chat_corpus/topic_classification_v4/embed_linear_small_white.json', 'w') as fwobj:
    for l in white:
        content = {
            'text':l[-1],
            'label':['正常']
        }
        fwobj.write(json.dumps(content, ensure_ascii=False)+'\n')

In [37]:
labels[0:10]

array([1, 1, 1, 1, 1, 1, 0, 1, 0, 0])

772017

384

In [ ]:

# def EmbedSimple(data, result_matrix):
    
    model = EmbedLinear(hidden_size=128, dropout_prob=0.1, num_labels=2)
    model.compile(method="multiclass", optimizer="Adam", learning_rate=0.001, momentum=0.0)
    
    votes = np.sum(result_matrix, axis=-1)
    labels = np.array(votes >= 3).astype(np.int)
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    
    for i in range(5):
        print('===start====', i)
        result = model.fit(data, labels)
        probs = model.predict_proba(result_matrix)
        print(probs.shape)
        labels = np.argmax(probs, axis=-1)
    # return probs, model





In [5]:


model = EmbedLinear(hidden_size=128, dropout_prob=0.1, num_labels=2)
model.compile(method="multiclass", optimizer="Adam", learning_rate=0.001, momentum=0.0)


from sklearn.ensemble import RandomForestClassifier
def SIMPLE(result_matrix):
    votes = np.sum(result_matrix, axis=-1)
    labels = np.array(votes >= 3).astype(np.int)
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    
    for i in range(5):
        print('===start====', i)
        result = clf.fit(result_matrix, labels)
        probs = clf.predict_proba(result_matrix)
        print(probs.shape)
        labels = np.argmax(probs, axis=-1)
    return probs, clf

In [3]:

result_matrix = np.array(result_matrix)
votes = np.sum(result_matrix, axis=-1)
(votes>=3).sum()

162260

In [6]:
probs, clf = SIMPLE(result_matrix)


===start==== 0
(772017, 2)
===start==== 1
(772017, 2)
===start==== 2
(772017, 2)
===start==== 3
(772017, 2)
===start==== 4
(772017, 2)


In [44]:
result_matrix.shape

(1279460, 8)